*OBjective: Find where can El Salvador citizens travel without the need of a visa, and provide recommendations based on different factors of living conditions.* 

**data from:** 

***https://visaguide.world/visa-free-countries/salvadoran-passport/***

***https://www.worlddata.info/cost-of-living.php***


Notes: found a better more resourceful website:
https://www.numbeo.com/cost-of-living/

In [23]:
import requests
import re
import pandas as pd
import numpy as np
import csv
from bs4 import BeautifulSoup

In [24]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.1234.56 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.google.com/',  # Replace with a commonly visited website
    'Connection': 'keep-alive',
}

r  = requests.get('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp', headers=headers)
r2 = requests.get('https://www.numbeo.com/cost-of-living/',headers=headers)
r3 = requests.get('https://visaguide.world/visa-free-countries/salvadoran-passport/',headers=headers)
column_soup  = BeautifulSoup(r.text, 'html.parser')
country_soup = BeautifulSoup(r2.text, 'html.parser')
visa_free_soup = BeautifulSoup(r3.text, 'html.parser')

data_for_column_names  = [tag for tag in column_soup.find_all(['th'])]
data_for_country_rows  = [tag for tag in country_soup.find_all(['h4','a'])]
es_visa_free_countries = [tag for tag in visa_free_soup.find_all(attrs={'ns_has-flag'})] 



In [25]:

print(f'r status code = {r.status_code}\nr2 status code = {r2.status_code}\nr3 status code = {r3.status_code}')

r status code = 200
r2 status code = 200
r3 status code = 200


In [26]:
columns_we_need           = [] # made an empty array bucket to store the scraped column names
country_rows              = [] # made an empty bucket for the countries
without_visa_countries    = [] 
evisa_countries           = []
visa_at_arrival_countries = []
visa_required_countries   = []
more_columns           = ['Visa Free Countries',
                          'eVisa Countries', 
                          'Visa on Arrival Countries',
                          'Visa Required Countries'
                          ]

# (column_names2) | adds names of columns we'll be using to empty array 
o = 0
while o < len(data_for_column_names):
    column_data = data_for_column_names[o].text
    o+=1
    columns_we_need.append(column_data)

# added more columns to insert values from Visaguide.world
start = 0
while start < len(more_columns):
    data = more_columns[start]
    columns_we_need.append(data)
    start+=1
    
# made the array bucket into a dictionary      
columns_we_need = {key: [] for key in columns_we_need}
 
# takes the soup data and gets the text from the arrays
# and stores the clean data into a new array
n = 65
while n < len(data_for_country_rows):
    country_data_in = data_for_country_rows[n].text

    if len(country_data_in) < 2:
        n+=1
        continue
    country_rows.append(country_data_in)
    n+=1

# (Country) adds values to dictionary 
country_rows = country_rows[:-11]
for value in country_rows:
    columns_we_need['Country'].append(value)


# (No Visa Countries) countries El Salvador can travel to without a visa
for i in range(len(es_visa_free_countries[:100])):
    visa_free_countries = es_visa_free_countries[i].text
    without_visa_countries.append(visa_free_countries)
for countries in without_visa_countries:
    columns_we_need['Visa Free Countries'].append(countries)


# (eVisa Countries) Countries that provide eVisa to Salvadoran Citizens
q = 100
while q < len(es_visa_free_countries[:128]):
    evisa = es_visa_free_countries[q].text
    evisa_countries.append(evisa)
    q+=1
for countries in evisa_countries:
    columns_we_need['eVisa Countries'].append(countries)

# (Visa upon Arrival) Countries that provide visa upon arrival
for s in range(128,158):
    visa_at_arrival_data = es_visa_free_countries[s].text
    visa_at_arrival_countries.append(visa_at_arrival_data)
    
for countries in visa_at_arrival_countries:
    columns_we_need['Visa on Arrival Countries'].append(countries)


# (Visa Required) Countries that require a visa to visit
for i in range(158,226):
    visa_req_data = es_visa_free_countries[i].text
    visa_required_countries.append(visa_req_data)
for countries in visa_required_countries:
    columns_we_need['Visa Required Countries'].append(countries)
    
# (Rank) Country Rank
# for ranks in columns_we_need['Country']:
    

In [27]:
columns_we_need.keys()

dict_keys(['Rank', 'Country', 'Quality of Life Index', 'Purchasing Power Index', 'Safety Index', 'Health Care Index', 'Cost of Living Index', 'Property Price to Income Ratio', 'Traffic Commute Time Index', 'Pollution Index', 'Climate Index', 'Visa Free Countries', 'eVisa Countries', 'Visa on Arrival Countries', 'Visa Required Countries'])

In [28]:
len(columns_we_need['Country'])

235

In [29]:

soup =  column_soup.find_all(['tr','td'])
soup[6:16]


[<td class="cityOrCountryInIndicesTable">Luxembourg</td>,
 <td style="text-align: right">200.1</td>,
 <td style="text-align: right">133.2</td>,
 <td style="text-align: right">66.0</td>,
 <td style="text-align: right">77.3</td>,
 <td style="text-align: right">71.7</td>,
 <td style="text-align: right">10.1</td>,
 <td style="text-align: right">25.4</td>,
 <td style="text-align: right">22.1</td>,
 <td style="text-align: right">82.6</td>]

In [43]:

empty = []
empty_clean = []
starting = 6
for i in range(1006):
    soup_data = soup[starting].text
    starting+=1
    empty.append(soup_data)
    if soup_data == '':
        empty.remove(soup_data)
   

for spot in range(len(empty)):
    if(spot+1) % 11 == 0:
        continue
    else:
        extract = empty[spot]
        empty_clean.append(extract)

for countries
print(empty)
print(empty_clean)

['Luxembourg', '200.1', '133.2', '66.0', '77.3', '71.7', '10.1', '25.4', '22.1', '82.6', '\n\nNetherlands\n200.1\n106.9\n73.8\n78.6\n67.5\n6.7\n23.8\n21.9\n86.7\n', 'Netherlands', '200.1', '106.9', '73.8', '78.6', '67.5', '6.7', '23.8', '21.9', '86.7', '\n\nIceland\n191.1\n100.4\n75.0\n67.2\n87.7\n5.9\n22.3\n15.7\n68.8\n', 'Iceland', '191.1', '100.4', '75.0', '67.2', '87.7', '5.9', '22.3', '15.7', '68.8', '\n\nDenmark\n190.6\n96.9\n73.4\n78.4\n79.2\n6.7\n28.6\n21.9\n81.8\n', 'Denmark', '190.6', '96.9', '73.4', '78.4', '79.2', '6.7', '28.6', '21.9', '81.8', '\n\nFinland\n188.1\n93.8\n73.5\n77.5\n68.7\n8.1\n26.4\n11.8\n56.6\n', 'Finland', '188.1', '93.8', '73.5', '77.5', '68.7', '8.1', '26.4', '11.8', '56.6', '\n\nSwitzerland\n185.8\n110.8\n75.1\n73.4\n117.3\n11.0\n30.2\n21.2\n79.6\n', 'Switzerland', '185.8', '110.8', '75.1', '73.4', '117.3', '11.0', '30.2', '21.2', '79.6', '\n\nOman\n184.8\n106.2\n80.4\n59.0\n49.6\n3.2\n22.3\n36.7\n67.2\n', 'Oman', '184.8', '106.2', '80.4', '59.0', '49.

In [ ]:
empty

NameError: name 'empty' is not defined

In [ ]:

len(empty)

922

['77.3', '71.7', '10.1', '25.4', '22.1', '82.6', '\n\nNetherlands\n200.1\n106.9\n73.8\n78.6\n67.5\n6.7\n23.8\n21.9\n86.7\n', '73.8', '6.7', '23.8', '21.9', '86.7', 'Iceland', '100.4', '67.2', '87.7', '22.3', '\n\nDenmark\n190.6\n96.9\n73.4\n78.4\n79.2\n6.7\n28.6\n21.9\n81.8\n', 'Denmark', '96.9', '73.4', '79.2', '6.7', '28.6', '\n\nFinland\n188.1\n93.8\n73.5\n77.5\n68.7\n8.1\n26.4\n11.8\n56.6\n', 'Finland', '73.5', '77.5', '8.1', '11.8', '56.6', '\n\nSwitzerland\n185.8\n110.8\n75.1\n73.4\n117.3\n11.0\n30.2\n21.2\n79.6\n', '110.8', '75.1', '73.4', '21.2', '79.6', '\n\nOman\n184.8\n106.2\n80.4\n59.0\n49.6\n3.2\n22.3\n36.7\n67.2\n', 'Oman', '106.2', '59.0', '3.2', '22.3', '67.2', 'Austria', '85.6', '72.4', '68.7', '23.8', '21.1', '\n\nNorway\n181.8\n90.7\n67.5\n76.6\n82.2\n8.2\n26.5\n18.0\n68.7\n', 'Norway', '90.7', '82.2', '8.2', '26.5', '68.7', 'Spain', '64.2', '77.5', '49.9', '7.1', '36.7', '93.7', '75.5', '75.9', '55.9', '10.2', '22.2', '17.1', '64.3', '176.0', '66.0', '10.2', '30.7',